# Data extraction and processing
see here for API tutorial on selecting data
http://docs.synapse.org/rest/org/sagebionetworks/repo/web/controller/TableExamples.html

In [ ]:
import pandas as pd
import json
import numpy as np
import pickle #to save files
from scipy.stats import skew, kurtosis, pearsonr
from scipy.signal import butter, welch, filtfilt, resample
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneGroupOut
import nolds
import time
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline

In [ ]:
from IPython.core.debugger import Tracer

In [ ]:
import synapseclient

In [ ]:
syn=synapseclient.login()

### Helper fcns - Feature extraction and clip generation

In [ ]:
def gen_clips(rawdata,clipsize=30,overlap=0,interp=True):
    
    len_tol = 0.8   #% of the intended clipsize below which clip is not used (deprecated for now)
    #reindex time (relative to start)
    t = rawdata.Timestamp
    t = t-t.iloc[0]
    rawdata.Timestamp = t
    #create clips data
    deltat = np.median(np.diff(rawdata.Timestamp))
    idx = np.arange(0,rawdata.Timestamp.iloc[-1],clipsize*(1-overlap))
    clips = []
    for i in idx:
        c = rawdata[(rawdata.Timestamp>=i) & (rawdata.Timestamp<i+clipsize)]
        if len(c) == 50*clipsize:
            clips.append(c)
        #resample clip if not enough samples are there
        elif len(c)>50*clipsize*len_tol: #for now hard code clip size (enforce 250 samples)
            t = c.Timestamp.values
            tnew = np.linspace(t[0],t[-1],50*clipsize)  
            x_res = resample(c.iloc[:,1].values,50*clipsize,t)[0]
            y_res = resample(c.iloc[:,2].values,50*clipsize,t)[0]
            z_res = resample(c.iloc[:,3].values,50*clipsize,t)[0]
            mag_res = np.sqrt(x_res**2+y_res**2+z_res**2)
            cres = pd.DataFrame(data=np.vstack((tnew,x_res,y_res,z_res,mag_res)).T,columns=
                                ['Timestamp', 'X', 'Y', 'Z', 'Mag'])
            clips.append(cres)
    return clips

def filterdata(rawdata,ftype='highpass',cutoff=0.75,cutoff_bp=[3,8],order=4):
    
    #takes rawdata as a parameter

    if not rawdata.empty:
        idx = rawdata.index
        idx = idx-idx[0]
        rawdata.index = idx
        x = rawdata.values
        #print(np.unique(np.diff(rawdata.index)))
        Fs = np.mean(1/(np.diff(rawdata.index))) #sampling rate
        if ftype != 'bandpass':
            #filter design
            cutoff_norm = cutoff/(0.5*Fs)
            b,a = butter(4,cutoff_norm,btype=ftype,analog=False)
        else:
            #filter design
            cutoff_low_norm = cutoff_bp[0]/(0.5*Fs)
            cutoff_high_norm = cutoff_bp[1]/(0.5*Fs)
            b,a = butter(order,[cutoff_low_norm,cutoff_high_norm],btype='bandpass',analog=False)

        #filter data
        xfilt = filtfilt(b,a,x,axis=0)
        rawdatafilt = pd.DataFrame(data=xfilt,index=rawdata.index,columns=rawdata.columns)
        return rawdatafilt

def power_spectra_welch(rawdata,fm,fM):
    #compute PSD on signal magnitude
    x = rawdata.iloc[:,-1]
    n = len(x) #number of samples in clip
    Fs = np.mean(1/(np.diff(x.index))) #sampling rate in clip
    f,Pxx_den = welch(x,Fs,nperseg=min(256,n))
    #return PSD in desired interval of freq
    inds = (f<=fM)&(f>=fm)
    f=f[inds]
    Pxx_den=Pxx_den[inds]
    Pxxdf = pd.DataFrame(data=Pxx_den,index=f,columns=['PSD_magnitude'])

    return Pxxdf


def feature_extraction(rawdata):
    
    features_list = ['RMSX','RMSY','RMSZ','rangeX','rangeY','rangeZ','meanX','meanY','meanZ','varX','varY','varZ',
                    'skewX','skewY','skewZ','kurtX','kurtY','kurtZ','Sen_X','Sen_Y','Sen_Z',
                    'xcor_peakXY','xcorr_peakXZ','xcorr_peakYZ','xcorr_lagXY','xcorr_lagXZ','xcorr_lagYZ',
                    'Dom_freqX','Pdom_relX','PSD_meanX','PSD_stdX','PSD_skewX','PSD_kurX',
                    'Dom_freqY','Pdom_relY','PSD_meanY','PSD_stdY','PSD_skewY','PSD_kurY',
                    'Dom_freqZ','Pdom_relZ','PSD_meanZ','PSD_stdZ','PSD_skewZ','PSD_kurZ',
                    'jerk_meanX','jerk_stdX','jerk_skewX','jerk_kurX',
                    'jerk_meanY','jerk_stdY','jerk_skewY','jerk_kurY',
                    'jerk_meanZ','jerk_stdZ','jerk_skewZ','jerk_kurZ',
                    'RMS_mag','range_mag','mean_mag','var_mag','skew_mag','kurt_mag','Sen_mag',
                    'Dom_freq_mag','Pdom_rel_mag','PSD_mean_mag','PSD_std_mag','PSD_skew_mag','PSD_kur_mag',
                    'jerk_mean_mag','jerk_std_mag','jerk_skew_mag','jerk_kur_mag']

    t = []
    t1 = time.time()
    rawdata_unfilt = rawdata.copy()

    Fs = np.mean(1/(np.diff(rawdata.index)/1000)) #sampling rate

    rawdata = filterdata(rawdata)
    t2 = time.time()
    t.append(t2-t1) #append shared preprocessing time


    t1 = time.time()
    #acceleration magnitude
    rawdata_wmag = rawdata_unfilt
    rawdata_wmag['Accel_Mag']=np.sqrt((rawdata_unfilt**2).sum(axis=1))
    t2 = time.time()
    t.append(t2-t1) #append magnitude computation time


    #extract features on current clip

    t1 = time.time()
    #Root mean square of signal on each axis
    N = len(rawdata)
    RMS = 1/N*np.sqrt(np.asarray(np.sum(rawdata**2,axis=0)))

    #range on each axis
    min_xyz = np.min(rawdata,axis=0)
    max_xyz = np.max(rawdata,axis=0)
    r = np.asarray(max_xyz-min_xyz)

    #Moments on each axis
    mean = np.asarray(np.mean(rawdata,axis=0))
    var = np.asarray(np.std(rawdata,axis=0))
    sk = skew(rawdata)
    kurt = kurtosis(rawdata)

    t2 = time.time()
    t.append(t2-t1) # append time domain features


    t1 = time.time()
    #sample entropy raw data (magnitude) and FFT
    sH_raw = []; #sH_fft = []

    for a in range(3):
        x = rawdata.iloc[:,a]
        n = len(x) #number of samples in clip
        Fs = np.mean(1/(np.diff(x.index))) #sampling rate in clip
        sH_raw.append(nolds.sampen(x)) #samp entr raw data
        #for now disable SH on fft
        # f,Pxx_den = welch(x,Fs,nperseg=min(256,n/4))
        # sH_fft.append(nolds.sampen(Pxx_den)) #samp entr fft

    t2 = time.time()
    t.append(t2-t1) # append Sen features time


    t1 = time.time()
    #Cross-correlation between axes pairs
    xcorr_xy = np.correlate(rawdata.iloc[:,0],rawdata.iloc[:,1],mode='same')
    # xcorr_xy = xcorr_xy/np.abs(np.sum(xcorr_xy)) #normalize values
    xcorr_peak_xy = np.max(xcorr_xy)
    xcorr_lag_xy = (np.argmax(xcorr_xy))/len(xcorr_xy) #normalized lag

    xcorr_xz = np.correlate(rawdata.iloc[:,0],rawdata.iloc[:,2],mode='same')
    # xcorr_xz = xcorr_xz/np.abs(np.sum(xcorr_xz)) #normalize values
    xcorr_peak_xz = np.max(xcorr_xz)
    xcorr_lag_xz = (np.argmax(xcorr_xz))/len(xcorr_xz)

    xcorr_yz = np.correlate(rawdata.iloc[:,1],rawdata.iloc[:,2],mode='same')
    # xcorr_yz = xcorr_yz/np.abs(np.sum(xcorr_yz)) #normalize values
    xcorr_peak_yz = np.max(xcorr_yz)
    xcorr_lag_yz = (np.argmax(xcorr_yz))/len(xcorr_yz)

    #pack xcorr features
    xcorr_peak = np.array([xcorr_peak_xy,xcorr_peak_xz,xcorr_peak_yz])
    xcorr_lag = np.array([xcorr_lag_xy,xcorr_lag_xz,xcorr_lag_yz])

    t2=time.time()
    t.append(t2-t1) # append xcorr computation time


    t1 = time.time()
    axes_F = np.array([])
    for a in range(3):
        x = rawdata.iloc[:,a]
        n = len(x) #number of samples in clip
        Fs = np.mean(1/(np.diff(x.index))) #sampling rate in clip
        f,Pxx_den = welch(x,Fs,nperseg=min(256,n))
        Pxx = pd.DataFrame(data=Pxx_den,index=f,columns=['PSD'])
        F_rel = np.asarray([Pxx.iloc[Pxx.index<12,-1].idxmax()])
        P_rel = Pxx.loc[F_rel].iloc[:,-1].values/Pxx.iloc[Pxx.index<12,-1].sum()
        F_moments = np.array([np.nanmean(Pxx.values),np.nanstd(Pxx.values),skew(Pxx.values)[0],kurtosis(Pxx.values)[0]])
        axes_F = np.concatenate((axes_F,F_rel,P_rel,F_moments))

    t2 = time.time()
    t.append(t2-t1) # append frequency axes computation time


    t1 = time.time()
    #moments of jerk axes
    axes_D = np.array([])
    for a in range(3):
        ax = rawdata.iloc[:,a].diff().values
        ax_moments = np.array([np.nanmean(ax),np.nanstd(ax),skew(ax[~np.isnan(ax)]),kurtosis(ax[~np.isnan(ax)])])
        axes_D = np.concatenate([axes_D,ax_moments])
    t2 = time.time()
    t.append(t2-t1) # append axes derivative computation time


    t1 = time.time()
    RMS_mag = 1/N*np.sqrt(np.sum(rawdata_wmag['Accel_Mag']**2,axis=0))
    r_mag = np.max(rawdata_wmag['Accel_Mag']) - np.min(rawdata_wmag['Accel_Mag'])
    mean_mag = np.mean(rawdata_wmag['Accel_Mag'])
    var_mag = np.std(rawdata_wmag['Accel_Mag'])
    sk_mag = skew(rawdata_wmag['Accel_Mag'])
    kurt_mag = kurtosis(rawdata_wmag['Accel_Mag'])
    t2 = time.time()
    t.append(t2-t1) # append magnitude time domain computation time


    t1 = time.time()
    sH_mag = nolds.sampen(rawdata_wmag['Accel_Mag'])
    t2 = time.time()
    t.append(t2-t1) # append magnitude entropy computation time


    t1 = time.time()
    #Dominant freq and relative magnitude (on acc magnitude)
    Pxx = power_spectra_welch(rawdata_wmag,fm=0,fM=Fs)
    domfreq = np.asarray([Pxx.iloc[Pxx.index<12,-1].idxmax()])
    Pdom_rel = Pxx.loc[domfreq].iloc[:,-1].values/Pxx.iloc[Pxx.index<12,-1].sum() #power at dominant freq rel to total

    #moments of PSD
    Pxx_moments = np.array([np.nanmean(Pxx.values),np.nanstd(Pxx.values),skew(Pxx.values)[0],kurtosis(Pxx.values)[0]])
    t2 = time.time()
    t.append(t2-t1) # append magnitude frequency computation time


    t1 = time.time()
    #moments of jerk magnitude
    jerk = rawdata_wmag['Accel_Mag'].diff().values
    jerk_moments = np.array([np.nanmean(jerk),np.nanstd(jerk),skew(jerk[~np.isnan(jerk)]),kurtosis(jerk[~np.isnan(jerk)])])
    t2 = time.time()
    t.append(t2-t1) # append magnitude derivative computation time

    #Assemble features in array
    Y = np.array([RMS_mag,r_mag,mean_mag,var_mag,sk_mag,kurt_mag,sH_mag])
    X = np.concatenate((RMS,r,mean,var,sk,kurt,sH_raw,xcorr_peak,xcorr_lag,axes_F,axes_D,Y,domfreq,Pdom_rel,Pxx_moments,jerk_moments))
    
    return X, features_list

In [ ]:
syntable = syn.tableQuery("SELECT * from syn20489608")
table = syntable.asDataFrame()
table.to_csv('Metadata.csv',index=False)

In [ ]:
syntable = syn.tableQuery("SELECT * from syn20489607")

In [ ]:
path = syn.downloadTableColumns(syntable,'smartwatch_accelerometer')

In [ ]:
table = syntable.asDataFrame()

In [ ]:
table['path']=table.smartwatch_accelerometer.astype(str).map(path)

In [ ]:
table.path.iloc[0]

In [ ]:
for row in table.iterrows():
    data = pd.read_csv(row[1].path)
    clips = gen_clips(data)
    # skip empty data
    if len(clips)==0:
        continue
    DF_Recording = pd.DataFrame()
    Fs=[]
    for c in clips:
        clip_edit = c.iloc[:,1:4].copy()
        clip_edit.index=c['Timestamp']
        F = feature_extraction(clip_edit)
        Fs.append(pd.DataFrame(data=np.reshape(F[0], (1, -1)),columns=F[1],index=[0]))
    
    D = pd.concat(Fs)
    D['ID'] = row[1].measurement_id
    
    D.to_csv(row[1].measurement_id+'.csv',index=False)

In [ ]:
Fs = os.listdir('.\\Features')

In [ ]:
D = pd.DataFrame()
for f in Fs:
    D_f = pd.read_csv('.\\Features\\'+f)
    D = pd.concat([D,D_f])

In [ ]:
D.to_csv('.\\Features_Full.csv',index=False)

In [ ]:
D.head()